In [10]:
from sklearn.neural_network import MLPRegressor
import numpy as np
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.datasets import make_blobs, make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import hvplot.pandas
import holoviews as hv

%matplotlib inline

read the GMA data with the areas and avg's for each
data contains; Actual sales price from the prio year and the pct change, the transactions from the prior year and the pct change, etc.


In [3]:


# Attempt to load the CSV file using a different encoding
data_csv = pd.read_csv(Path("./GMA_data.csv"))

data_csv.head()



,Geo Market Area,ASP 12/2021 - 12/2022,ASP 12/2022 - 12/2023,% Change,Transactions 12/2021 - 12/2022,Transactions 12/2022 - 12/2023,% Change.1,MOI 11/2022,MOI 11/2023,% Change.2,Avg DOM 12/2021 - 12/2022,Avg DOM 12/2022 - 12/2023,% Change.3
0,1960/Cypress,"$305,022","$319,320",4.70%,290,205.0,-29.30%,1.5,2.6,72.10%,15.0,35.7,138.00%
1,1960/Cypress Creek North,"$321,283","$331,624",3.20%,411,245.0,-40.40%,2.2,4.6,107.50%,27.6,39.2,42.00%
2,1960/Cypress Creek South,"$265,879","$258,609",-2.70%,786,613.0,-22.00%,2.2,2.9,28.20%,23.3,40.2,72.50%
3,Aldine Area,"$237,692","$237,042",-0.30%,880,755.0,-14.20%,2.5,2.9,14.60%,23.2,38.1,64.20%
4,Alief,"$260,950","$262,756",0.70%,523,443.0,-15.30%,2.5,3.2,25.50%,21.8,31.6,45.00%


In [4]:
data_csv.dtypes


Geo Market Area                    object
ASP 12/2021 - 12/2022              object
ASP 12/2022 - 12/2023              object
% Change                           object
Transactions 12/2021 - 12/2022      int64
Transactions 12/2022 - 12/2023    float64
% Change.1                         object
MOI 11/2022                        object
MOI 11/2023                        object
% Change.2                         object
Avg DOM 12/2021 - 12/2022         float64
Avg DOM 12/2022 - 12/2023         float64
% Change.3                         object
dtype: object

Obtain and read the (general) houston data and retrive head with name of data 
dataset contains ; dollar volume, average price, and median price.

In [6]:
H_data_csv = pd.read_csv(Path("./Houston_data.csv"))

H_data_csv.head()   

                

,Unnamed: 0,Closed,% Change,Unnamed: 3,% Change.1,Average,% Change.2,Median,% Change.3
0,Property Type,Listings*,Year Ago,Dollar Volume,Year Ago,Price,Year Ago,Price,Year Ago
1,All (New and Existing),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Single Family,"7,983",-4%,"$3,353,852,000",-1%,"$420,124",3%,"$339,000",0%
3,Townhouse/Condominium,556,-19%,"$145,065,478",-17%,"$260,909",3%,"$225,000",3%
4,Multifamily,65,-22%,"$31,514,889",-27%,"$484,844",-6%,"$390,000",-5%


In [13]:
H_data_csv.dtypes

Unnamed: 0    object
Closed        object
% Change      object
Unnamed: 3    object
% Change.1    object
Average       object
% Change.2    object
Median        object
% Change.3    object
dtype: object

give the user to grab a name of a area.
slice into the dict "Geograpgocal area"

In [8]:
# Prompt the user to enter an area name
user_input_area = input("Enter the name of the area: ")

# Filter the DataFrame for rows where the 'Geo Market Area' matches the user input
data_csv = data_csv[data_csv['Geo Market Area'] == user_input_area]

# Display the filtered data
print(data_csv)


Empty DataFrame
Columns: [Geo Market Area, ASP 12/2021 - 12/2022, ASP 12/2022 - 12/2023, % Change, Transactions 12/2021 - 12/2022, Transactions 12/2022 - 12/2023, % Change.1, MOI 11/2022, MOI 11/2023, % Change.2, Avg DOM 12/2021 - 12/2022, Avg DOM 12/2022 - 12/2023, % Change.3]
Index: []


grab the days on market data and make a dataframe for the average DOM

In [14]:
Dom_data = pd.read_csv(Path("./Days_on_market.csv"))
Dom_data.head()

,Unnamed: 0,Price/,% Change,Unnamed: 3,% Change.1,Pending,% Change.2,New,% Change.3,Active,%Change
0,Property Type,Sqft*,Year Ago,DOM,Year Ago,Listings,Year Ago,Listings,Year Ago,Listings,Year Ago
1,All (New and Existing),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Single Family,$175,0%,51,55%,"62,184",-9%,"93,700",-4%,"20,614",41%
3,Townhouse/Condominium,$167,0%,42,20%,"4,559",-21%,"7,161",-12%,"1,396",23%
4,Multifamily,$171,-1%,56,8%,541,-11%,"1,288",5%,407,44%


In [ ]:
market_days = 

(optional) step: grab and scale data and concat the data values to make a new datframe.


In [15]:
Dom_data.dtypes

Unnamed: 0    object
Price/        object
% Change      object
Unnamed: 3    object
% Change.1    object
Pending       object
% Change.2    object
New           object
% Change.3    object
Active        object
%Change       object
dtype: object

Grab the Historic data of August: along with the year and year befores data, number of listings, avg/median pricig, and (optional:DOM)
concat the avg and medean pct to the new data frame dataframe, obtain the dtypes


In [16]:
Historic_pricing = pd.read_csv(Path("./Historic_pricing.csv"))

Historic_pricing.head()

,Unnamed: 0,Unnamed: 1,Closed,Average,Median,Price/,New,Active,Pending,Unnamed: 9,Close to
0,Year,Month,Listings*,Price,Price,Sqft,Listings,Listings,Listings,DOM,Orig. List Price
1,2021,Jan,"4,815","$321,238","$255,000",$132,"7,202","9,088","6,097",43,96.70%
2,2021,Feb,"4,671","$348,928","$270,000",$139,"6,148","7,781","5,721",42,97.20%
3,2021,Mar,"7,327","$363,397","$285,000",$143,"8,831","7,184","7,889",39,97.80%
4,2021,Apr,"7,459","$372,590","$290,500",$148,"10,164","7,725","8,195",33,98.80%


In [17]:
Historic_pricing.dtypes

Unnamed: 0    object
Unnamed: 1    object
Closed        object
Average       object
Median        object
Price/        object
New           object
Active        object
Pending       object
Unnamed: 9    object
Close to      object
dtype: object

In [21]:
Hist_df = Historic_pricing['Average']
Hist_df.head()

0        Price
1    $321,238 
2    $348,928 
3    $363,397 
4    $372,590 
Name: Average, dtype: object